In [1]:
import qiskit
qiskit.__qiskit_version__

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-16c9f27add4e>", line 1, in <module>
    import qiskit
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\qiskit\__init__.py", line 27, in <module>
    from qiskit.circuit import ClassicalRegister
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\qiskit\circuit\__init__.py", line 214, in <module>
    from .quantumcircuit import QuantumCircuit
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\qiskit\circuit\quantumcircuit.py", line 25, in <module>
    import numpy as np
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\numpy\__init__.py", line 140, in <module>
    from . import core
  File "C:\Users\Saesun Kim\anaconda3\envs\Qiskit\lib\site-packages\numpy\core\__init__.py", line 72,

TypeError: object of type 'NoneType' has no len()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.optim import LBFGS, SGD,Adam 

from qiskit  import Aer, QuantumCircuit
from qiskit.utils import QuantumInstance
from qiskit.opflow import AerPauliExpectation
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.neural_networks import CircuitQNN, TwoLayerQNN
from qiskit_machine_learning.connectors import TorchConnector

import torch.optim as optim
from torch.nn import (Module, Conv2d, Linear, Dropout2d, NLLLoss,
                     MaxPool2d, Flatten, Sequential, ReLU)

qi = QuantumInstance(Aer.get_backend('statevector_simulator'))

In [ ]:
import torch
torch.cuda.is_available()

# Data Test 0

In [ ]:
import numpy as np
from numpy import pi
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.nn import (Module, Conv2d, Linear, Dropout2d, NLLLoss,
                     MaxPool2d, Flatten, Sequential, ReLU)

data0Path = r'../../../dataset/data3c.txt'
data0Label = r'../../../dataset/data3clabel.txt'

dataCoords = np.loadtxt(data0Path)
dataLabels = np.loadtxt(data0Label)

# Make a data structure which is easier to work with
# for shuffling. 
# Also, notice we change the data labels from {0, 1} to {-1, +1}
data1 = list(zip(dataCoords, 2*dataLabels-1))

In [ ]:
def binary(x):
    return ('0'*(4-len('{:b}'.format(x) ))+'{:b}'.format(x))
def firsttwo(x):
    return x[:2]
parity = lambda x: firsttwo(binary(x)).count('1') % 2

In [ ]:
from pandas.core.common import flatten
import torch

np.random.seed(2)
#data_ixs = np.random.choice(len(shuffled_data), size=len(shuffled_data))
data_ixs = np.random.choice(len(data1), size=100)

X= [np.array(list(flatten([data1[j][0],data1[j][0]]))) for j in data_ixs]
y = [data1[j][1] for j in data_ixs]
y01 =  [ (x + 1)/2 for x in y]
X_ = Tensor(X)
y_ = Tensor(y).reshape(len(y), 1)
y01_ = Tensor(y01).reshape(len(y)).long()

num_inputs=4;

feature_map = QuantumCircuit(4, name='Embed')
feature_map.rx(Parameter('x[0]'),0)
feature_map.rx(Parameter('x[1]'),1)
feature_map.rx(Parameter('x[2]'),2)
feature_map.rx(Parameter('x[3]'),3)
feature_map.ry(pi/4,0)
feature_map.ry(pi/4,1)
feature_map.ry(pi/4,2)
feature_map.ry(pi/4,3)
feature_map.rz(pi/4,0)
feature_map.rz(pi/4,1)
feature_map.rz(pi/4,2)
feature_map.rz(pi/4,3)


param_y=[];
for i in range(12):
    param_y.append((Parameter('θ'+str(i))))
ansatz = QuantumCircuit(4, name='PQC')
for i in range(4):
    ansatz.rx(param_y[i],i)
for i in range(4):
    ansatz.rz(param_y[i+4],i)

qc = QuantumCircuit(num_inputs)
qc.append(feature_map, range(num_inputs))
qc.append(ansatz, range(num_inputs))

ansatz.draw('mpl')

# SGD optimizer

In [ ]:
learningR=[round(((i+1)/10)**2,2) for i in range(20)]

    
output_shape=2;
epochs = 10     # set number of epochs

model=[];
lossLR=[];



for l in learningR:
    np.random.seed(2)  
    qnn2 = CircuitQNN(qc, input_params=feature_map.parameters, weight_params=ansatz.parameters, 
                      interpret=parity, output_shape=output_shape, quantum_instance=qi)
    initial_weights = 0.1*(2*np.random.rand(qnn2.num_weights) - 1)
    # set up PyTorch module
    model2 = TorchConnector(qnn2, initial_weights)
    # define optimizer and loss function
    optimizer = optim.SGD(model2.parameters(),lr=l)
    f_loss = MSELoss(reduction='mean')

    # start training
    model2.train()   # set model to training mode
    
    print("Learning Rate is ", l)
    # define objective function
    for epoch in range(epochs):
        optimizer.zero_grad()        # initialize gradient
        loss = 0.0                                             # initialize loss    
        for x, y_target in zip(X, y01):                        # evaluate batch loss
            output = model2(Tensor(x)).reshape(1, 2)           # forward pass
            targets=Tensor([y_target]).long()
            targets = targets.to(torch.float32)
            #targets = targets.unsqueeze(1)
            loss += f_loss(output, targets) 
        loss.backward()                              # backward pass
        print("__Loss is ",loss.item())                           # print loss

        # run optimizer
        optimizer.step() 
    model.append(model2)
    lossLR.append(loss.item())


In [ ]:
ln=len(learningR)
fig = plt.figure(figsize=(15, 10))
accLR=[];
for lr in range(ln):

    ax = fig.add_subplot(ln, 2, 2*lr+1)
    y_predict = []
    for x in X:
        output = model[lr](Tensor(x))
        y_predict += [np.argmax(output.detach().numpy())]
    acc=sum(y_predict == np.array(y01))/len(np.array(y01))
    print('Learning Rate:', learningR[lr],'loss:',lossLR[lr],' Accuracy:', acc)
    accLR.append(acc)
    # plot results
    # red == wrongly classified
    for x, y_target, y_ in zip(X, y01, y_predict):
        if y_target == 1:
            ax.plot(x[0], x[1], 'bo')
        else:
            ax.plot(x[0], x[1], 'go')
        if y_target != y_:
            ax.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=2)


    ax = fig.add_subplot(ln, 2, 2*lr+2)
    for x, y_target, y_ in zip(X, y01, y_predict):
        if y_target == 1:
            ax.plot(x[0], x[1], 'bo')
        else:
            ax.plot(x[0], x[1], 'go')

    X1 = np.linspace(0, 1, num=10)
    Z1 = np.zeros((len(X1), len(X1)))

    # Contour map
    for j in range(len(X1)):
        for k in range(len(X1)):
            # Fill Z with the labels (numerical values)
            # the inner loop goes over the columns of Z,
            # which corresponds to sweeping x-values
            # Therefore, the role of j,k is flipped in the signature
            Z1[j, k] = np.argmax(model[lr](Tensor([X1[k],X1[j],X1[k],X1[j]])).detach().numpy())

    ax.contourf(X1, X1, Z1, cmap='bwr', levels=30)

In [ ]:
f1=fig
f1.set_size_inches(10, 80)
f1.savefig('SGD_3c.png')

In [ ]:
lossLR50=[x/50 for x in lossLR]
plt.plot(learningR,lossLR50, label = "Loss/50")
plt.plot(learningR,accLR, label = "Accuracy")
plt.xlabel('Learning Rate')
plt.legend()
plt.show()

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
lvA=[[learningR[x],accLR[x]] for x in range(len(accLR))]
np.savetxt("SGD_Accuracy_3c.txt",lvA,fmt='%.2f')